In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-Machine-Learning-Project-with-MLFlow\\End-to-end-Machine-Learning-Project-with-MLflow\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'd:\\End-to-End-Machine-Learning-Project-with-MLFlow\\End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

#Let therebe display from multiple commands

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
test=pd.read_csv("artifacts\data_ingestion\Test.csv")

In [7]:
test.isnull().sum()  # Checking null values

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [8]:
test['Item_Weight'].fillna(test['Item_Weight'].mean(),inplace=True)  # Filling missing values with mean for Item weight column for test.csv

In [9]:
test['Item_Weight'].fillna(test['Item_Weight'].mean(),inplace=True)  # Filling missing values with mean for Item weight column for test.csv

In [10]:
test.Outlet_Size.value_counts()   # Balancing categories with less category value by filling missing with "High category"

Outlet_Size
Medium    1862
Small     1592
High       621
Name: count, dtype: int64

In [11]:
test['Outlet_Size'].fillna("High", inplace=True)

In [12]:
test.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [13]:
test["Item_Fat_Content"].value_counts()  # Similar data correction required for test file

Item_Fat_Content
Low Fat    3396
Regular    1935
LF          206
reg          78
low fat      66
Name: count, dtype: int64

In [14]:
test["Item_Fat_Content"] = test["Item_Fat_Content"].replace(['LF'], 'Low Fat')
test["Item_Fat_Content"] = test["Item_Fat_Content"].replace(['low fat'], 'Low Fat')
test["Item_Fat_Content"] = test["Item_Fat_Content"].replace(['reg'], 'Regular')

In [15]:
test["Item_Fat_Content"].value_counts() #Replacement of column category is done with filling correct category and data correction is complete now

Item_Fat_Content
Low Fat    3668
Regular    2013
Name: count, dtype: int64

In [16]:
test['Years_Established'] = test['Outlet_Establishment_Year'].apply(lambda x: 2024 - x)
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Years_Established
0,FDW58,20.750000,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,25
1,FDW14,8.300000,Regular,0.038428,Dairy,87.3198,OUT017,2007,High,Tier 2,Supermarket Type1,17
2,NCN55,14.600000,Low Fat,0.099575,Others,241.7538,OUT010,1998,High,Tier 3,Grocery Store,26
3,FDQ58,7.315000,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,High,Tier 2,Supermarket Type1,17
4,FDY38,12.695633,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,39


In [17]:
test.drop(['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [18]:
# Preprocessing Task before Model Building
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [19]:
test['Item_Fat_Content']= le.fit_transform(test['Item_Fat_Content'])
test['Item_Type']= le.fit_transform(test['Item_Type'])
test['Outlet_Size']= le.fit_transform(test['Outlet_Size'])
test['Outlet_Location_Type']= le.fit_transform(test['Outlet_Location_Type'])
test['Outlet_Type']= le.fit_transform(test['Outlet_Type'])

In [20]:
import joblib

In [21]:
# load the model from disk
loaded_model = joblib.load("artifacts/model_trainer/model.joblib")

In [22]:
 Z= loaded_model.predict(test)

In [23]:
Z

array([7.279404 , 7.122663 , 6.464471 , ..., 7.3910193, 7.991502 ,
       7.068426 ], dtype=float32)

In [24]:
np.exp([Z])   # Bringing into Original value of Target column

array([[1450.1235, 1239.7474,  641.9247, ..., 1621.3579, 2955.7324,
        1174.2985]], dtype=float32)

In [25]:
# first define function
def storesales_default(Item_Weight: float, Item_Fat_Content: int, Item_Visibility: float, Item_Type: int, Item_MRP: float,Outlet_Establishment_Year: int,   Outlet_Size: int,   Outlet_Location_Type:int,   Outlet_Type:int,    Years_Established:int)-> float:

  """
  Predict the probability of loan default using a pre-trained machine learning pipeline.

  Args:
      Item_Weight (float): Weight of Product.
      Item_Fat_Content (int): Product is low fat or high fat.
      Item_Visibility (float): % of total display area in store allocated to product.
      Item_Type (int): product category.
      Item_MRP (float): amount of product.
      Outlet_Establishment_Year (int): Store established year.
      Outlet_Size (int): Store Size.
      Outlet_Location_Type (int): Type of city Store belongs.
      Outlet_Type (int): Grocery store or Supermarket.
      Years_Established (int): How old the Store is.

  Returns:
      float: Predict the Sales of per product and per Store
  """

  return loaded_model.predict(pd.DataFrame([[Item_Weight, Item_Fat_Content, Item_Visibility, Item_Type, Item_MRP, Outlet_Establishment_Year, Outlet_Size, Outlet_Location_Type, Outlet_Type, Years_Established ]],
                                             columns = ['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Years_Established']))

In [26]:
storesales_default(20.750000,   0,  0.007565,   13, 107.8622,   1999,   1,  0,  1,  25)

array([7.279404], dtype=float32)

In [27]:


# Measure inference time
start_time = time.time()
storesales_default(20.750000,   0,  0.007565,   13, 107.8622,   1999,   1,  0,  1,  25)
end_time = time.time()

inference_time = end_time - start_time
print(f'Inference Time: {inference_time:.4f} seconds')


array([7.279404], dtype=float32)

Inference Time: 0.0090 seconds
